# Skript für den Jetbot der Roadfollowing und Collision Avoidance kombiniert

In [ ]:


import ipywidgets as widgets
import torch
import torchvision
import traitlets
from IPython.display import display
from torch import Tensor, functional

from jetbot import Camera, bgr8_to_jpeg
from lib.ExtendedRobot import ExtendedRobot, Handel, ReturnData, ReturnCommand


Load models

In [ ]:
model_road = torchvision.models.resnet18(pretrained=False)
model_road.fc = torch.nn.Linear(512, 2)
model_road.load_state_dict(torch.load('best_steering_model_xy.pth'))

model_collision = torchvision.models.alexnet(pretrained=False)
model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, 2)
model_collision.load_state_dict(torch.load('best_model.pth'))

models = {
    "road": model_road,
    "collision": model_collision
}


Add cuda device to models

In [ ]:
device = torch.device('cuda')
model_road = model_road.to(device)
model_collision = model_collision.to(device)

Create Camera instance

In [ ]:
camera = Camera.instance(width=224, height=224, fps=10)

In [ ]:
robot: ExtendedRobot = ExtendedRobot(camera, models, device)

Widget for information and camera feed display

In [ ]:
image_widget = widgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

Sliders and information display

In [ ]:
#Collision Avoidance sliders
blocked_slider = widgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')

toggle_exec_btn = widgets.ToggleButton(
    value=False,
    description='Start',
    disabled=False,
    button_style='success',
    tooltip='Description',
    icon='check'
)
display(toggle_exec_btn)

display(image_widget, blocked_slider)


Variables for slider controls

In [ ]:
class HandelCollision(Handel):

    def get_collision_chance(self, model, image) -> float:
        collision = model(image)
        collision_softmax = functional.softmax(collision, dim=1).flatten()
        f: float = float(collision_softmax[0])
        blocked_slider.value = f
        return f

    def execute(self, models: {}, image: any, tensor: Tensor, previous_values: list) -> ReturnData:

        f = self.get_collision_chance(models["collision"], image)
        if f > 0.8:
            return ReturnData(command=ReturnCommand.STOP)
        else:
            return ReturnData(command=ReturnCommand.CONTINUE)


robot.register(HandelCollision())

In [ ]:
class HandelRoadFollowing(Handel):

    def get_road_direction(self, model, tensor: Tensor) -> (float, float):
        model_xy = model(tensor).detach().float().cpu().numpy().flatten()
        model_x = model_xy[0]
        model_y = (0.5 - model_xy[1]) / 2.0
        return model_x, model_y

    def execute(self, models: {}, image: any, tensor: Tensor, previous_values: list) -> ReturnData:
        x, y = self.get_road_direction(models["road"], tensor)
        return ReturnData(command=ReturnCommand.CONTINUE, poi=(x, y))

robot.register(HandelRoadFollowing())

In [ ]:

def start_stop_toggle(obj):
    if obj["new"]:
        toggle_exec_btn.button_style = 'warning'
        toggle_exec_btn.icon = 'warning'
        toggle_exec_btn.description = "Stop"
        robot.start()
    else:
        toggle_exec_btn.button_style = 'success'
        toggle_exec_btn.icon = 'check'
        toggle_exec_btn.description = 'Start'
        robot.stop()


toggle_exec_btn.observe(start_stop_toggle, 'value')